In [1]:
import re
import csv
from pyspark import *
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [2]:
sc = SparkContext("local")
sqlContext = SparkSession.builder.getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/11 13:23:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
def str2float(input):
    try:
        return float(input)
    except:
        return None

udf_str2float = udf(str2float, FloatType())

# convert text file to text, tag and score list

In [5]:
def convert_file_to_text_and_tag_list(
    emotion_tag,
    data_file,
    ):
    schema = StructType()\
        .add("tweet_id",StringType(),True)\
        .add("text",StringType(),True)\
        .add("emotion",StringType(),True)\
        .add("intensity",FloatType(),True)
    train = sqlContext.read.format('csv')\
        .options(delimiter='\t')\
        .schema(schema)\
        .load(data_file)\
        .withColumn("intensity", udf_str2float("intensity"))
    train.registerTempTable("train")
    train_list = sqlContext.sql(u"""
        SELECT *, 
        CASE 
            WHEN emotion = '%s' THEN 1
            ELSE 0
        END AS label
        FROM train
        WHERE emotion IS NOT NULL
        """%(emotion_tag)).collect()
    texts = [r.text for r in train_list]
    tags = [r.label for r in train_list]
    return texts, tags

In [6]:
def convert_file_to_text_and_score_list(
    emotion_tag,
    data_file,
    ):
    udf_str2float = udf(str2float, FloatType())
    schema = StructType()\
        .add("tweet_id",StringType(),True)\
        .add("text",StringType(),True)\
        .add("emotion",StringType(),True)\
        .add("intensity",FloatType(),True)
    train = sqlContext.read.format('csv')\
        .options(delimiter='\t')\
        .schema(schema)\
        .load(data_file)\
        .withColumn("intensity", udf_str2float("intensity"))
    train.registerTempTable("train")
    train_list = sqlContext.sql(u"""
        SELECT *
        FROM train
        WHERE emotion = '%s' AND intensity IS NOT NULL
        """%(emotion_tag)).collect()
    texts = [r.text for r in train_list]
    scores = [r.intensity for r in train_list]
    return texts, scores